# Opik Optimizer (beta)

Welcome to the Opik Opitimizer beta program! In this notebook we'll walk through a basic examples of a number of an optimizer algorithm.

## Setup

First, let's install the needed Python packages.

We'll of course need `opik`:

In [1]:
%pip install opik --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.6 MB/s eta 0:00:00


Next, we'll install the beta version of the `opik-optimizer`:

In [4]:
!pip install git+https://github.com/comet-ml/opik#subdirectory=sdks/opik_optimizer --upgrade --quiet > pip-install.log 2>&1

[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

You can use your own workspace.

In [1]:
import opik

# Configure Opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /home/dsblank/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [2]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To optimize any prompt, we'll need:

1. A starting prompt
2. A metric
3. A dataset

For this initial test, we'll start with a portion of the HotpotQA dataset.

HotpotQA is a question/answering dataset featuring natural questions, with correct answers. It was collected by a team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal.

Let's take a look at the demo dataset "hotpot-300".

First, we get or create it. This will add it to our default Opik project:

In [3]:
from opik_optimizer.demo import get_or_create_dataset
from opik_optimizer.demo.cache import get_litellm_cache


opik_dataset = get_or_create_dataset("hotpot-300")
get_litellm_cache("test")

Inserted 0 record(s) in litellm cache


Let's look at a specific row from the dataset:

In [4]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0195d400-517b-7f17-b746-ff3d084463ac',
 'question': 'Were both drinks, the Smoking Bishop and the Mickey Slim, popular in different countries?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [5]:
rows[2]

{'id': '0195d400-5179-7259-8104-1f0a55a13ac2',
 'question': 'Woody Wuthrie wrote the song "Do Re Mi" which was about migrants and their experiences when they arrive in California and are greeted with severe storms that came in waves in what years?',
 'answer': '1934, 1936, and 1939–1940'}

As you can see, the answers can be a little messy. We'll need a good metric to able to determine whether a prompt is good or not. But let's start simple.

We'll use the `Equals` metric from Opik.

It works like this:

In [6]:
from opik.evaluation.metrics import Equals

metric = Equals()

metric.score("Hello", "Hello")

OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0196ab60-35a2-7453-98b9-eaf8e07d4300&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='equals_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

Here we can see that value is 1.0. That meens that the two values are the same.

What counts as equal? 

In [7]:
metric.score("Hello", "heLLo")

ScoreResult(name='equals_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [8]:
metric.score("hello", "hell")

ScoreResult(name='equals_metric', value=0.0, reason=None, metadata=None, scoring_failed=False)

Ok, we have a dataset and a metric, now we are ready to construct an optimizer.

We can use an OpenAI model name, or more generally, a LiteLLM model name. Just make sure you have your model API key set as we did above.

We'll start with the `FewShotBayesianOptimizer`:

In [9]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
    MetricConfig,
    from_llm_response_text,
    from_dataset_field,
    TaskConfig,
)

optimizer = FewShotBayesianOptimizer(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

Now, we need a prompt to optimize. Given the examples above, let's try something like:

In [10]:
initial_prompt = "Provide an answer to the question"

In [11]:
project_name = "optimize-few-shot-hotpot-300"

In [12]:
messages = [
    {"role": "system", "content": initial_prompt},
    {"role": "user", "content": "{{question}}"},
]

In [13]:
metric_config = MetricConfig(
    metric=Equals(project_name=project_name),
    inputs={
        "output": from_llm_response_text(),
        "reference": from_dataset_field(name="answer"),
    },
)

task_config = TaskConfig(
    instruction_prompt=initial_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)

In [14]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    prompt=messages,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot-300 (100 samples) ──╮
│                             │
│ Total time:        00:00:27 │
│ Number of samples: 100      │
│                             │
│ equals_metric: 0.0000 (avg) │
│                             │
╰─────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=533225;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ab60-4326-7217-8360-7b3b820cd0a7&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.0

The score 0.15 is pretty low. Let's see if we can optimize it!

The FewShotBayesianOptimizer can, fairly quickly, create better prompts.

Let's try it out. It takes exactly the same parameters as evaluate_prompt(), but will run for a minute or so.

In [15]:
result = optimizer.optimize_prompt(opik_dataset, metric_config, task_config, n_trials=3, n_samples=100)

[I 2025-05-07 11:32:42,943] A new study created in memory with name: no-name-99b18981-8714-4cbb-b177-c2a667cea925


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot-300 (100 samples) ──╮
│                             │
│ Total time:        00:00:13 │
│ Number of samples: 100      │
│                             │
│ equals_metric: 0.1100 (avg) │
│                             │
╰─────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=988712;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ab60-b1b3-7b58-8af2-617866cb7aa6&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

...

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot-300 (100 samples) ──╮
│                             │
│ Total time:        00:00:10 │
│ Number of samples: 100      │
│                             │
│ equals_metric: 0.2700 (avg) │
│                             │
╰─────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=510311;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ab61-15ef-7bfa-9e5d-c9bab90a9f7a&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

[I 2025-05-07 11:33:20,229] Trial 2 finished with value: 0.27 and parameters: {'n_examples': 6, 'example_0': 76, 'example_1': 205, 'example_2': 11, 'example_3': 42, 'example_4': 84, 'example_5': 262}. Best is trial 2 with value: 0.27.


In [16]:
for message in result.prompt:
    print(message)

{'role': 'system', 'content': 'Provide an answer to the question\n\nYou are an intelligent assistant that learns from few-shot examples provided earlier in the conversation. Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies, using them as a guide'}
{'role': 'user', 'content': '\n{\n  "question": "In what country was televison introduced in 1957 and TV6, a terrestrial satellite and cable television channel launched?"\n}\n'}
{'role': 'assistant', 'content': 'Lithuania'}
{'role': 'user', 'content': '\n{\n  "question": "What did the first book of Gary Zukav explore ?"\n}\n'}
{'role': 'assistant', 'content': 'empirical topics in modern physics research'}
{'role': 'user', 'content': '\n{\n  "question": "What is the nationality of the author of Progress in Optics?"\n}\n'}
{'role': 'assistant', 'content': 'American'}
{'role': 'user', 'content': '\n{\n  "question": "Where is the head office of the company that developed No issue, lelo tissue lo

Although there is some randomness, you probably got a better prompt. My result is about 5 times better.

This is just the initial framework for optimizers for Opik!

Please see the [Opik Optimizer repo](https://github.com/comet-ml/opik/tree/main/sdks/opik_optimizer) for additional examples using different algorithms.